In [ ]:
from torch import nn, Tensor
import torch.nn as nn
import torch
import torch.nn.functional as F
from safetensors.torch import load_file

from dataclasses import dataclass
from typing import Optional,  Tuple
from einops import rearrange
from transformers import AutoModelForCausalLM, AutoTokenizer


In [ ]:
# TODO code load dataset and dataloader
# TODO code training loop
# TODO code evaluation loop
# TODO add support to tensorboard
# TODO add weights initialization

# validation

In [1]:
from tinyllama import TinyLlama, name_to_config, load_model_weights
tiny_LLaMA_1b = TinyLlama(name_to_config['tiny_LLaMA_1b'])

In [ ]:
load_model_weights(tiny_LLaMA_1b, "model/models--TinyLlama--TinyLlama-1.1B-Chat-v1.0/snapshots/fe8a4ea1ffedaf415f4da2f062534de366a451e6/model.safetensors")

In [5]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,  map_device="auto", add_eos_token=True, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    cache_dir="model",
    # attn_implementation=attn_implementation
)

In [6]:
inputs_ids = tokenizer("Hello, how are you?", return_tensors="pt").input_ids

In [ ]:
out1 = tiny_LLaMA_1b(inputs_ids)
out2 = model(inputs_ids, output_attentions=True)

In [8]:
assert (out1 == out2.logits).all(), "The model output is different from the reference model output"

In [ ]:
chat = [
  {'role': 'user', 'content': 'Hello, how are you?'},
]

inputs_ids = tokenizer.apply_chat_template(chat, return_tensors="pt", tokenize=True, add_generation_prompt=True)

In [ ]:
out = tiny_LLaMA_1b.generate(inputs_ids, max_length=10, sample=False)

In [ ]:
tokenizer.decode(out[0])

# Dataset

In [13]:
from huggingface_hub import hf_hub_download
from huggingface_hub import HfFileSystem
import requests

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the variables
hf_token = os.getenv("HF_TOKEN")

In [20]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,  map_device="auto", add_eos_token=True, use_fast=True)

## SlimPajama

In [15]:
def download_folder(model_id, folder_path, repo_type="model", output_dir="data"):

    for i in range(50):
        file_name = f"/example_train_{i}.jsonl.zst"
        hf_hub_download(repo_id=model_id, filename=(f'{folder_path}{file_name}'), local_dir=output_dir, repo_type=repo_type)


In [7]:
download_folder("cerebras/SlimPajama-627B", "train/chunk1", "dataset",  output_dir="pre-processing/data/slimpajama")

In [ ]:
import os
import pandas as pd
import zstandard as zstd


def read_zst_jsonl(file_path):
    with open(file_path, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        
        decompressed_data = dctx.stream_reader(f)
        return pd.read_json(decompressed_data, lines=True)


folder_path = 'data/slimpajama/train/chunk1'


files = [f for f in os.listdir(folder_path) if f.endswith('.jsonl.zst')]


df = pd.concat([read_zst_jsonl(os.path.join(folder_path, file)) for file in files], ignore_index=True)


df.shape

In [21]:
df = df.sample(50000, random_state=42, ignore_index=True)

In [39]:
df['input_ids'] = df['text'].apply(lambda x: tokenizer(x, return_tensors="pt").input_ids[0].tolist())

In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)

hf_dataset.push_to_hub("slimpajama", token=hf_token)

In [46]:
df['input_ids_size'] = df['input_ids'].apply(lambda x: len(x))

In [ ]:
sum(df['input_ids_size'])

## StarCoder 

In [ ]:
url = "https://huggingface.co/datasets/bigcode/starcoderdata/resolve/main/python/train-00000-of-00059.parquet"
output_path = "pre-processing/data/train-00000-of-00059.parquet"

headers = {"Authorization": f"Bearer {hf_token}"}
response = requests.get(url, headers=headers, stream=True)

if response.status_code == 200:
    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"File successfully downloaded: {output_path}")

In [ ]:

df = pd.read_parquet("pre-processing/data/train-00000-of-00059.parquet")
df.shape

In [53]:
df = df.sample(10000, random_state=42, ignore_index=True)

In [54]:
df['input_ids'] = df['content'].apply(lambda x: tokenizer(x, return_tensors="pt").input_ids[0].tolist())

In [67]:
df['input_ids_size'] = df['input_ids'].apply(lambda x: len(x))

In [ ]:
sum(df['input_ids_size'])

In [ ]:
hf_dataset = Dataset.from_pandas(df)
hf_dataset.push_to_hub("starcoder", token=hf_token)

## DataLoaders